In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

from sm_utils import *

In [2]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [3]:
df = process_lr_metadata(config['lr']['meta'], ['human', 'mouse'])

In [14]:
# species=['human', 'mouse']

# # lr stuff
# for i,s in enumerate(species):
#     f = expand(config['lr']['meta'], species=s)[0]
#     if i == 0:
#         lr_df = pd.read_csv(f, sep='\t')
#         lr_df['species'] = s
#     else:
#         temp = pd.read_csv(f, sep='\t')
#         temp['species'] = s
#         lr_df = pd.concat([lr_df, temp], axis=0)
        
# # for i, f in enumerate(expand(config['lr']['meta'], species=species)):
# #     if i == 0:
# #         lr_df = pd.read_csv(f, sep='\t')
# lr_meta = process_encode_metadata(expand(config['lr']['encode_meta'], species=species)[0])


In [4]:
df.species.unique()

array(['human', 'mouse'], dtype=object)

In [6]:
df.head()

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,document_urls,document_labels,platform,RIN,spikeins,species,age,sex,genotype,matching_human_samples
0,ENCSR989ZYL,a673_1_1,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,https://www.encodeproject.org/documents/6d583a...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.8,True,human,NaN,NaN,NaN,NaN
1,ENCSR989ZYL,a673_1_2,a673,A673,a673,bone,NaN,cell_line,#de3700,NaN,...,https://www.encodeproject.org/documents/6d583a...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.8,True,human,NaN,NaN,NaN,NaN
2,ENCSR081NRO,adrenal_gland_1_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,https://www.encodeproject.org/documents/54012f...,'non-size selected cDNA libraries for use in P...,Pacific Biosciences Sequel,9.8,False,human,NaN,NaN,NaN,NaN
3,ENCSR563RLX,adrenal_gland_2_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,https://www.encodeproject.org/documents/3baa46...,"'PacBio libraries v3 (October, 2020) Protocol ...",Pacific Biosciences Sequel II,NaN,True,human,NaN,NaN,NaN,NaN
4,ENCSR995WKW,adrenal_gland_3_1,adrenal_gland,Adrenal gland,adrenal gland,adrenal gland,NaN,tissue,#e69b00,"adrenal_gland,adrenal_10d,adrenal_14d,adrenal_...",...,https://www.encodeproject.org/documents/54012f...,'non-size selected cDNA libraries for use in P...,Pacific Biosciences Sequel,9.4,False,human,NaN,NaN,NaN,NaN


In [7]:
df.tail()

,ENCODE_experiment_id,dataset,sample,sample_display,general_tissue_cell_type,fig1_tissue_label,health_status,tissue_or_cell_line,sample_color_hex_code,matching_mouse_samples,...,document_urls,document_labels,platform,RIN,spikeins,species,age,sex,genotype,matching_human_samples
122,ENCSR909NPF,hippocampus_2mo_f_1,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
123,ENCSR909NPF,hippocampus_2mo_f_2,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
124,ENCSR972DID,hippocampus_2mo_m_1,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,m,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
125,ENCSR972DID,hippocampus_2mo_m_2,hippocampus_2mo,Hippocampus PNM2,hippocampus,hippocampus,NaN,tissue,#c96579,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,2mo,m,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"
126,ENCSR038FOD,hippocampus_10d_f_1,hippocampus_10d,Hippocampus,hippocampus,hippocampus,NaN,tissue,NaN,NaN,...,https://www.encodeproject.org/documents/bf543f...,'ENCODE Long Read RNA-Seq Analysis Protocol fo...,Pacific Biosciences Sequel II,9.0,True,mouse,10d,f,b6/cast,"['brain', 'pgp1_excite_neuron', 'pgp1_astro']"


In [18]:
len(lr_df.loc[lr_df.species=='mouse'])

126